# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [ ]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from anthropic import Anthropic
import gradio as gr



In [ ]:
load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
claude_api_key = os.getenv('CLAUDE_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

if claude_api_key:
    print(f"Claude API Key exists and begins {claude_api_key[:8]}")
else:
    print("Claude API Key not set")



openai = OpenAI()
claude = Anthropic(api_key=claude_api_key)


In [ ]:
system_message = """
You are a helpful assistant that can answer questions about the code. and give a detailed explanation. and example.
your response should be in markdown format. and well structured.
"""

MODELS = {
    "GPT": "gpt-4.1-mini",
    "Claude": "claude-haiku-4-5-20251001",
}

In [ ]:


def put_message_in_chatbot(message, history):
    return "", history + [{"role": "user", "content": message}]

def process_audio(audio_filepath, history):
    if audio_filepath is None:
        return history, None
    text = transcribe(audio_filepath)
    history = history + [{"role": "user", "content": text}]
    return history, None

def chat(history, model_name):
    messages = [{"role": h["role"], "content": h["content"]} for h in history]
    model_id = MODELS[model_name]

    if model_name == "Claude":
        reply = ''
        with claude.messages.stream(
            model=model_id,
            messages=messages,
            max_tokens=1024,
            system=system_message,
        ) as stream:
            for text in stream.text_stream:
                reply += text
                yield history + [{"role": "assistant", "content": reply}]

        history += [{"role": "assistant", "content": reply}]
    else:
        full_messages = [{"role": "system", "content": system_message}] + messages
        response = openai.chat.completions.create(model=model_id, messages=full_messages, stream=True)
        reply = ''
        for chunk in response:
            if chunk.choices[0].finish_reason == "stop":
                break
            if chunk.choices[0].delta.content is not None:
                reply += chunk.choices[0].delta.content
                yield history + [{"role": "assistant", "content": reply}]

        history += [{"role": "assistant", "content": reply}]

def transcribe(audio_filepath):
    with open(audio_filepath, "rb") as audio_file:
        result = openai.audio.transcriptions.create(
            model="whisper-1",
            file=audio_file,
        )
    return result.text

with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=500, type="messages")
    with gr.Row():
        message = gr.Textbox(label="Chat with our AI Assistant:")
        audio_input = gr.Audio(
            sources=["microphone", "upload"],
            type="filepath",
            label="Or speak your question",
        )
    with gr.Row():
        model_selector = gr.Dropdown(
            choices=list(MODELS.keys()),
            value="GPT",
            label="Select Model",
        )

    message.submit(
        put_message_in_chatbot, inputs=[message, chatbot], outputs=[message, chatbot]
    ).then(
        chat, inputs=[chatbot, model_selector], outputs=chatbot
    )

    audio_input.change(
        process_audio, inputs=[audio_input, chatbot], outputs=[chatbot, audio_input]
    ).then(
        chat, inputs=[chatbot, model_selector], outputs=chatbot
    )

ui.launch(inbrowser=True)
